Create local Client

In [70]:
from dask.distributed import Client
client = Client()
client


C:\Users\eleme\.conda\envs\teamx\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50208 instead
  warnings.warn(
C:\Users\eleme\.conda\envs\teamx\Lib\site-packages\distributed\client.py:1612: VersionMismatchWarning: Mismatched versions found

+-------------+----------+-----------+----------+
| Package     | Client   | Scheduler | Workers  |
+-------------+----------+-----------+----------+
| dask        | 2025.3.0 | 2025.3.0  | 2025.5.1 |
| distributed | 2025.3.0 | 2025.3.0  | 2025.5.1 |
+-------------+----------+-----------+----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


UndefinedError: 'n_workers' is undefined

<Client: 'tcp://127.0.0.1:50211' processes=4 threads=8, memory=7.89 GiB>

In [71]:
client = Client('127.0.0.1:50211')

In [72]:
import sys
from operator import concat

from xarray import decode_cf

sys.path.append("D:/MSc_Arbeit/model_comparison_codes")
import confg
import xarray as xr
import numpy as np
from functools import partial
import metpy.calc as mpcalc
from metpy.units import units
import datetime
from pathlib import Path

In [73]:
lat_ibk = 47.259998
lon_ibk = 11.384167

%%time
icon_10 = xr.open_dataset(confg.icon_folder_3D + "/ICON_20171015_latlon.nc")
# 
# icon_point = icon_point.compute()

In [80]:
%%time
icon_100 = xr.open_dataset(confg.icon_folder_3D + "/ICON_20171015_latlon.nc", 
                            chunks={'lat': 10, 'lon': 10})

CPU times: total: 62.5 ms
Wall time: 74.6 ms


In [75]:
%%time
icon_point = icon_10.sel(lat=lat_ibk, lon=lon_ibk, method="nearest")

CPU times: total: 31.2 ms
Wall time: 101 ms


In [81]:
%%time
icon_point = icon_100.sel(lat=lat_ibk, lon=lon_ibk, method="nearest")

CPU times: total: 15.6 ms
Wall time: 25.6 ms


In [82]:
def convert_calc_variables(ds):
    """
    Converts and calculates meteorological variables for a xarray Dataset.

    Parameters:
    - df: A xarray Dataset containing the columns 'p' for pressure in Pa
          and 'th' for potential temperature in Kelvin.

    Returns:
    - A xarray Dataset with the original data and new columns:
      'pressure' in hPa and 'temperature' in degrees Celsius.
    """

    # Convert pressure from Pa to hPa
    ds['pressure'] = (ds['pres'] / 100.0) * units.hPa

    # calc pot temp
    ds["th"] = mpcalc.potential_temperature(ds['pressure'], ds["temp"] * units.kelvin)

    # convert temp to °C
    ds["temp"]  = (ds["temp"] - 273.15) * units.degC

    return ds.metpy.dequantify()  # remove units from the dataset

In [83]:
%%time
icon_calc = convert_calc_variables(icon_point)

CPU times: total: 46.9 ms
Wall time: 199 ms


In [84]:
%%time
icon_point_100 = icon_calc.compute()  # takes insanely long! >300s!

CPU times: total: 2min 14s
Wall time: 9min 1s


In [18]:

%%time
import dask.array as da
x = da.random.random((10000, 10000, 10), chunks=(1000, 1000, 5))
y = da.random.random((10000, 10000, 10), chunks=(1000, 1000, 5))
z = (da.arcsin(x) + da.arccos(y)).sum(axis=(1, 2))
z.persist()



CPU times: total: 141 ms
Wall time: 270 ms


dask.array<sum-aggregate, shape=(10000,), dtype=float64, chunksize=(1000,), chunktype=numpy.ndarray>